In [1]:
import cv2
from imageio import imread,imwrite
import os
import numpy as np
from matplotlib import pyplot as plt
import xml.sax
from tqdm import tqdm_notebook as tqdm

### Creating Parser 

Parser parses xml files rescaling input to (for h,w =608)), output should be normalized

In [31]:
#normalizing is done here
_GLOBAL_HEIGHT=720
_GLOBAL_WIDTH=1280
class LotHandler(xml.sax.ContentHandler):
    def __init__(self):
        self.all=[] # [occupied,cx,cy,w,h]
        self.all4=[] # [cx,cy,w,h]]
    #call when an element starts
    def startElement(self,tag,attributes):
        self.CurrentData=tag
        if (tag == 'space'):
#             print(attributes.__getitem__('occupied'))
            key=attributes.getNames()[-1]
            self.all.append(attributes.__getitem__(key))
        if(tag == 'center'):
            x=attributes['x']
            y=attributes['y']
            x=float(x)/_GLOBAL_WIDTH*608
            y=float(y)/_GLOBAL_HEIGHT*608
            x=str(x)
            y=str(y)
            
            self.all.append(x)
            self.all.append(y)
            self.all4.append(x)
            self.all4.append(y)
        if(tag == 'size'):
            w=attributes['w']
            h=attributes['h']
            w=float(w)/_GLOBAL_WIDTH*608
            h=float(h)/_GLOBAL_HEIGHT*608
            w=str(w)
            h=str(h)
            self.all.append(w)
            self.all.append(h)
            self.all4.append(w)
            self.all4.append(h)
    def getData(self,as_int=False):
        """
        self.all = [id,cx,cy,w,h]
        self.all4 = [cx,cy,w,h]
        
        """
        
        if as_int:
            return list(map(lambda x : int(float(x)),self.all)),list(map(lambda x : int(float(x)),self.all4))
        return self.all,self.all4
    

In [15]:
parser=xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
Handler = LotHandler()
parser.setContentHandler( Handler )
doc_path='1.xml'
parser.parse(doc_path)

id 67


In [ ]:
details,params=Handler.getData()

In [ ]:
text_file_name='./1.txt'
f=open(text_file_name,'w+')
for i,x in enumerate(details):
    if (i+1)%5 == 0:
        f.write(x)
        f.write('\n')
    else:
        f.write(x)
        f.write(" ")
f.close()

## Plotting


In [ ]:
_,params=Handler.getData(as_int=True)

In [3]:
def centreToCorner(centre_params):
    """
    Params : cx,cy,w,h
    return : lx,ly,rx,ry
    """
    cx,cy,w,h=centre_params
    lx=cx-w/2
    ly=cy+h/2
    rx=cx+w/2
    ry=cy-h/2
    corner=[lx,ly,rx,ry]
    corner=list(map(int,corner))
    return corner

def plotLots(image,params_list):
    """
    params_list [cx,cy,w,h]
    """
    img=image.copy()
    for i in range(len(params_list)):
        if(i%4==0):
            cx=params_list[i + 0]
            cy=params_list[i + 1]
            w=params_list[i + 2]
            h=params_list[i + 3]
            params=[cx,cy,w,h]
            lx,ly,rx,ry=centreToCorner(params)
            cv2.rectangle(img,(lx,ly),(rx,ry),(0,255,0),2)
    return img

In [ ]:
img=imread('./1.jpg')

In [ ]:
plt.imshow(img)

In [ ]:
img=cv2.resize(img,(608,608))
plt.imshow(plotLots(img,params))

# EVERTHING SHOULD BE SCALED TO 608


## Creating Annotations.txt

In [24]:
a=[1,2,3,4,5,6]
for x in zip( *[iter(a)]*3) :
    print(x)

(1, 2, 3)
(4, 5, 6)


In [36]:
image_width=608
image_height=608
# image list
images_dir=os.path.abspath('./data/image_annotations/')
doc_dir=os.path.abspath('./data/image_annotations/')
target_dir=os.path.abspath('./data/image_annotations_txt/')
error_list=[]
images_list=[]
_boxesWithoutStat=0
_totalBoxes=0
## read annotations from xml and outputs normalized list
for file in tqdm(os.listdir(images_dir)):
    filename,extension=file.split('.')
    
    if(extension=='jpg'):
        doc_path=doc_dir+'/'+filename+'.xml'
        parser=xml.sax.make_parser()
        parser.setFeature(xml.sax.handler.feature_namespaces, 0)
        Handler = LotHandler()
        parser.setContentHandler( Handler )
#         try:
        parser.parse(doc_path)
        details,params=Handler.getData(as_int=False)
        
        # some ids does not have occupancy info removing thos
        details_mod=[]
        for batch in zip(*[iter(details)]*5):
            if(int(batch[0])>1):
                _boxesWithoutStat+=1
                _totalBoxes+=1
            else:
                details_mod.extend(batch)
                _totalBoxes+=1
        details=details_mod
        #writing to file
        textfile=target_dir+'/'+filename+'.txt'
        f=open(textfile,'w+')
        for i,x in enumerate(details):
            if(i%5)!=0:
                x=float(x)/608 ## normalzing
                x=str(x)
            if (i+1)%5 == 0:
                f.write(x)
                f.write('\n')
            else:
                f.write(x)
                f.write(" ")
        f.close()
#         #writing resized image
#         image_file_path=target_dir+'/'+filename+'.jpg'
#         img=imread(images_dir+'/'+filename+'.jpg')
#         img=cv2.resize(img,(608,608))
#         imwrite(image_file_path,img)
#         except:
#             print(f"Error at {filename}")
#             error_list.append(filename)
print("Completed")
print(f"Perncentage of boxes without parking info {_boxesWithoutStat*100/_totalBoxes} ")

Completed
Perncentage of boxes without parking info 3.5457966889405275 


In [33]:
len(details)

200

## Sanity check

In [ ]:
def centreToCorner(centre_params):
    """
    Params : cx,cy,w,h
    return : lx,ly,rx,ry
    """
    cx,cy,w,h=centre_params
    lx=cx-w/2
    ly=cy+h/2
    rx=cx+w/2
    ry=cy-h/2
    corner=[lx,ly,rx,ry]
    corner=list(map(int,corner))
    return corner

def plotLots(image,params_list):
    """
    params_list [cx,cy,w,h]
    
    expects denormalized input
    """
    img=image.copy()
    for i in range(len(params_list)):
        if(i%4==0):
            cx=params_list[i + 0]
            cy=params_list[i + 1]
            w=params_list[i + 2]
            h=params_list[i + 3]
            params=[cx,cy,w,h]
            lx,ly,rx,ry=centreToCorner(params)
            cv2.rectangle(img,(lx,ly),(rx,ry),(0,255,0),2)
#             cv2.rectangle(img,(rx,ry),(lx,ly),(0,255,0),2)
            
    return img

In [ ]:
images_dir=os.path.abspath('./data/image_annotations_txt/')
images_list=[]
for file in tqdm(os.listdir(images_dir)):
    filename,extension=file.split('.')
    if(extension=='jpg'):
        doc_path=images_dir+'/'+filename+'.jpg'
        images_list.append(doc_path)

In [ ]:
img=imread(images_list[0])

In [ ]:
file_path='/media/aghinsa/DATA/workspace/ParkingLot/data/image_annotations_txt/2012-09-11_15_16_58.txt'

In [ ]:
box_params=[]
with open(file_path) as f:
    for line in f.readlines():
        line=line.rstrip("\n")
        line=line.split(" ")[1:]
        box_params.extend(list(map(lambda x :(float(x)),line)))

In [ ]:
box_params=list(map(lambda x:int(x*608),box_params))

In [ ]:
box_params[:4]

In [ ]:
plt.imshow(img)

In [ ]:
plotted=plotLots(img,box_params_denorm)

In [ ]:
plt.imshow(plotted)

###  Train Test Split


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
images_dir=os.path.abspath('./data/image_annotations_txt/')
images_list=[]
for file in tqdm(os.listdir(images_dir)):
    filename,extension=file.split('.')
    if(extension=='jpg'):
        doc_path=images_dir+'/'+filename+'.jpg'
        images_list.append(doc_path)

In [ ]:
train_list,test_list=train_test_split(images_list,test_size=0.2,shuffle=True)

In [ ]:
f=open('parkLot_train.txt','w+')
for file in train_list:
    f.write(file)
    f.write("\n")
f.close()

f=open('parkLot_test.txt','w+')
for file in test_list:
    f.write(file)
    f.write("\n")
f.close()